In [1]:
import pandas as pd
pd.options.display.max_colwidth = 200

import seaborn as sns
sns.set(rc={'figure.figsize':(15,5)})

import matplotlib.pyplot as plt
import numpy as np

In [2]:
merchant_agg = pd.read_csv("../data/curated/clusters/input/agg_transaction_train.csv")[['merchant_abn', 'total_number_of_distinct_customers', 
    'monthly_average_number_of_orders', 'monthly_average_bnpl_revenue']]
merchant_clusters = pd.read_csv("../data/curated/clusters/output/merchant_clusters.csv")

display(merchant_agg.head(3))
display(merchant_clusters.head(3))

,merchant_abn,total_number_of_distinct_customers,monthly_average_number_of_orders,monthly_average_bnpl_revenue
0,10023283211,2619,154.333333,33301.586223
1,10342410215,729,40.888889,15624.420327
2,10346855916,7,1.400000,2186.800497


,merchant_abn,label
0,10023283211,1
1,10342410215,2
2,10346855916,0


**Merge datasets**

In [3]:
merged_feature_cluster = merchant_clusters.merge(merchant_agg, on='merchant_abn', how='inner').drop('merchant_abn', axis=1)
display(merged_feature_cluster.head(3))

,label,total_number_of_distinct_customers,monthly_average_number_of_orders,monthly_average_bnpl_revenue
0,1,2619,154.333333,33301.586223
1,2,729,40.888889,15624.420327
2,0,7,1.400000,2186.800497


**Perfrom log transformation**

In [4]:
agg_byCluster = pd.DataFrame(np.log(merged_feature_cluster.iloc[:,1:])).add_prefix('log_')
agg_byCluster['cluster'] = pd.Categorical(merged_feature_cluster.label)

display(agg_byCluster.head(3))
display(agg_byCluster.groupby('cluster').describe().T)

,log_total_number_of_distinct_customers,log_monthly_average_number_of_orders,log_monthly_average_bnpl_revenue,cluster
0,7.870548,5.039115,10.413360,1
1,6.591674,3.710858,9.656590,2
2,1.945910,0.336472,7.690195,0


cluster                                                0           1  \
log_total_number_of_distinct_customers count  924.000000  824.000000   
                                       mean     3.041300    8.656969   
                                       std      1.094127    0.634698   
                                       min      0.000000    7.590347   
                                       25%      2.484907    8.101071   
                                       50%      3.295837    8.550047   
                                       75%      3.828641    9.131622   
                                       max      5.087596   10.089137   
log_monthly_average_number_of_orders   count  924.000000  824.000000   
                                       mean     0.713308    5.981607   
                                       std      0.497299    0.871671   
                                       min      0.000000    4.755026   
                                       25%      0.316379    5.282710   
                                       50%      0.693147    5.777480   
                                       75%      1.026897    6.473118   
                                       max      2.254383    9.529497   
log_monthly_average_bnpl_revenue       count  924.000000  824.000000   
                                       mean     7.806826   10.747525   
                                       std      0.760012    0.988169   
                                       min      5.122598    7.719359   
                                       25%      7.237132   10.222586   
                                       50%      7.753828   10.780184   
                                       75%      8.327079   11.088514   
                                       max      9.910631   13.056325   

cluster                                                 2  
log_total_number_of_distinct_customers count  2251.000000  
                                       mean      5.944994  
                                       std       1.005133  
                                       min       3.637586  
                                       25%       5.144579  
                                       50%       5.940171  
                                       75%       6.792344  
                                       max       7.922624  
log_monthly_average_number_of_orders   count  2251.000000  
                                       mean      3.067234  
                                       std       1.016369  
                                       min       0.747214  
                                       25%       2.257123  
                                       50%       3.060271  
                                       75%       3.923073  
                                       max       5.080714  
log_monthly_average_bnpl_revenue       count  2251.000000  
                                       mean      8.703137  
                                       std       1.323906  
                                       min       6.150718  
                                       25%       7.548150  
                                       50%       8.727291  
                                       75%       9.716200  
                                       max      12.730122

## Plot the Marginal Distances

In [5]:
def marginal_error(x_variable, agg_byCluster=agg_byCluster):
    '''this function creates plots of marginal distances between each cluster with respect to each feature'''
    sns.scatterplot(data=agg_byCluster, hue='cluster', style='cluster', x=x_variable, y=agg_byCluster['cluster'], palette='deep')

    # compute boundaries for clusters, outliers are removed using 1.5IQR
    description = agg_byCluster.groupby('cluster')[x_variable].describe().T
    description.loc["lb",:] = description.loc["25%"]-1.5*(description.loc["75%"]-description.loc["25%"])
    description.loc["ub",:] = description.loc["75%"]+1.5*(description.loc["75%"]-description.loc["25%"])
    description.loc["chosen_lower"] = np.where(description.loc["lb",:] < description.loc["min",:], description.loc["min",:], description.loc["lb",:])
    description.loc["chosen_upper"] = np.where(description.loc["ub",:] < description.loc["max",:], description.loc["ub",:], description.loc["max",:])

    # visualizing the boundaries
    colour = ['b', 'orange', 'green']
    cluster = [0,1,2]
    for i in range(0,3):
        plt.axvline(x =  description.loc["chosen_lower",cluster[i]], color = colour[i], linestyle = ':')
        plt.axvline(x =  description.loc["chosen_upper",cluster[i]], color = colour[i], linestyle = ':')
        
    # plot the marginal distances
    cluster_sequence = description.loc['chosen_lower',:].T.sort_values().index.to_list()
    
    for i in [0,1]:
        x_value = description.loc["chosen_upper",cluster_sequence[i]]-(description.loc["chosen_upper",cluster_sequence[i]]-description.loc["chosen_lower", cluster_sequence[i+1]])/2
        plt.axvline(x = x_value, color = 'r', label = f'MoE_{cluster_sequence[i],cluster_sequence[i+1]} ≈ {round(x_value,2)}')
        
    plt.legend()
    # plt.savefig(f'../plots/{x_variable}.jpg')
    # plt.clf()
    
    return